In [1]:
import os
import click
import string
import numpy as np
from tqdm import tqdm
from models.model_loader import load_model
from torchvision.transforms import Compose
from dataset.data_transform import Resize, Rotation, Translation, Scale
from dataset.test_data import TestDataset
from dataset.text_data import TextDataset
from dataset.collate_fn import text_collate
from lr_policy import StepLR

import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
from torch import Tensor
from torch.utils.data import DataLoader
from warpctc_pytorch import CTCLoss

from test import test
import json
import cv2

In [2]:
data_path = '../../dataset/newspaper_words/'

In [3]:
fin = open(data_path + 'desc.json', 'rb')
data = json.load(fin)
fin.close()

In [4]:
seq_proj = [10,20]
backend = 'resnet18'
snapshot = '../../weights/newspaper_word_crnn_reduced_classes/crnn_resnet18_84_best'
cuda = True
net = load_model(data['abc'], seq_proj, backend, snapshot, cuda)
net = net.cuda(0)

/usr/local/lib/python2.7/dist-packages/torch/nn/parallel/data_parallel.py:24: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 1. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


In [5]:
for d in data['train']:
    img = cv2.imread(data_path + 'data/' + d['name'])
    img = cv2.resize(img, (320,32))
    img = np.transpose(img, (2,0,1))
    img = Variable(torch.from_numpy(np.asarray([img])).type(torch.FloatTensor)).cuda(0)
    out = net(img, decode=True)
    print out.size()
    break

TypeError: forward() takes at least 2 arguments (2 given)

tensor([[[ 7.7626e-10,  8.1577e-16,  8.6331e-14,  ...,  4.0548e-15,
           7.6462e-17,  1.6312e-13]],

        [[ 1.0000e+00,  3.5099e-16,  1.5557e-15,  ...,  8.0020e-17,
           9.0254e-17,  1.4222e-15]],

        [[ 9.9724e-01,  7.0021e-15,  4.9437e-15,  ...,  1.7566e-18,
           6.4344e-19,  1.7124e-16]],

        ...,

        [[ 9.9977e-01,  3.7130e-14,  1.0813e-12,  ...,  1.3727e-16,
           3.0512e-14,  1.7372e-14]],

        [[ 5.6329e-03,  1.0722e-11,  2.4740e-08,  ...,  1.2267e-12,
           1.3377e-08,  3.0798e-10]],

        [[ 2.2175e-02,  6.4801e-11,  1.2510e-08,  ...,  6.7660e-10,
           2.0158e-06,  1.4631e-08]]], device='cuda:0')